In [1]:
import numpy as np
import json
import vtk
from vtk.util import numpy_support
import os


DIM = 3
DIM_DELIM = {0:'', 1:'$', 2:'%'}

def ch2val(c):  # Analogue to native Lenia conversion
    if c in '.b': return 0
    elif c == 'o': return 255
    elif len(c) == 1: return ord(c) - ord('A') + 1
    else: return (ord(c[0]) - ord('p')) * 24 + (ord(c[1]) - ord('A') + 25)

def append_stack(stack, val, count, is_repeat):
    if is_repeat:
        stack.extend([val] * int(count or 1))
    else:
        for _ in range(int(count or 1)):
            stack.append(val[:])

def recur_get_max_lens(dim, arr, max_lens):
    if isinstance(arr, list):
        max_lens[dim] = max(max_lens[dim], len(arr))
        if len(arr) > 0 and isinstance(arr[0], list):
            for sub_arr in arr:
                recur_get_max_lens(dim + 1, sub_arr, max_lens)

def recur_cubify(dim, arr, max_lens):
    if dim == len(max_lens) - 1:
        while len(arr) < max_lens[dim]:
            arr.append(0)
    else:
        for sub_arr in arr:
            recur_cubify(dim + 1, sub_arr, max_lens)
        while len(arr) < max_lens[dim]:
            arr.append([0] * max_lens[dim + 1])

def rle2arr(st):
    stacks = [[] for _ in range(DIM)]
    last, count = '', ''
    delims = list(DIM_DELIM.values())
    st = st.rstrip('"') + DIM_DELIM[DIM-1]
    
    for ch in st:
        if ch.isdigit():
            count += ch
        elif ch in 'pqrstuvwxy@':
            last = ch
        else:
            if last + ch not in delims:
                append_stack(stacks[0], ch2val(last + ch), count, is_repeat=True)
            else:
                dim = delims.index(last + ch)
                for d in range(dim):
                    append_stack(stacks[d + 1], stacks[d], count, is_repeat=False)
                    stacks[d] = []
            last, count = '', ''
    
    A = stacks[DIM-1]
    max_lens = [0 for _ in range(DIM)]
    recur_get_max_lens(0, A, max_lens)
    recur_cubify(0, A, max_lens)
    return np.asarray(A)

def load_json_and_convert_to_numpy(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    cells_data = data['cells']
    
    # Combine all RLE strings into a single string
    combined_rle_string = ''.join(cells_data)
    
    # Process the combined RLE string to get a 3D array
    decoded_array = rle2arr(combined_rle_string)
    
    return decoded_array

def save_array_to_vtk(data_3d, filename):
    nx, ny, nz = data_3d.shape

    image = vtk.vtkImageData()
    image.SetDimensions(nx, ny, nz)
    image.SetSpacing(1, 1, 1)
    
    vtk_data_array = numpy_support.numpy_to_vtk(num_array=data_3d.ravel(order='F'), deep=True, array_type=vtk.VTK_FLOAT)

    image.GetPointData().SetScalars(vtk_data_array)
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(image)
    writer.Write()

In [68]:
for i in range(31, 32): # Still needs missing case consideration
    for j in range(1, 1000): # Automatically number of images for i-th image
        i_s = str(i)
        j_s = str(j)

        file_path = 'save\\' + i_s + '\\' + i_s + '_' + j_s + '.json'
        save_path = 'vtk\\' + i_s + '\\' + i_s + '_' + j_s + '.vti'
        print(file_path)
        numpy_array = load_json_and_convert_to_numpy(file_path)
        print(save_path)
        save_array_to_vtk(numpy_array, save_path)

save\31\31_1.json
vtk\31\31_1.vti
save\31\31_2.json
vtk\31\31_2.vti
save\31\31_3.json
vtk\31\31_3.vti
save\31\31_4.json
vtk\31\31_4.vti
save\31\31_5.json
vtk\31\31_5.vti
save\31\31_6.json
vtk\31\31_6.vti
save\31\31_7.json
vtk\31\31_7.vti
save\31\31_8.json
vtk\31\31_8.vti
save\31\31_9.json
vtk\31\31_9.vti
save\31\31_10.json
vtk\31\31_10.vti
save\31\31_11.json
vtk\31\31_11.vti
save\31\31_12.json
vtk\31\31_12.vti
save\31\31_13.json
vtk\31\31_13.vti
save\31\31_14.json
vtk\31\31_14.vti
save\31\31_15.json
vtk\31\31_15.vti
save\31\31_16.json
vtk\31\31_16.vti
save\31\31_17.json
vtk\31\31_17.vti
save\31\31_18.json
vtk\31\31_18.vti
save\31\31_19.json
vtk\31\31_19.vti
save\31\31_20.json
vtk\31\31_20.vti
save\31\31_21.json
vtk\31\31_21.vti
save\31\31_22.json
vtk\31\31_22.vti
save\31\31_23.json
vtk\31\31_23.vti
save\31\31_24.json
vtk\31\31_24.vti
save\31\31_25.json
vtk\31\31_25.vti
save\31\31_26.json
vtk\31\31_26.vti
save\31\31_27.json
vtk\31\31_27.vti
save\31\31_28.json
vtk\31\31_28.vti
save\31\31

In [69]:
def get_bounding_box(vtk_files):
    """Calculate the overall bounding box for a list of VTK files."""
    min_bounds = np.array([np.inf, np.inf, np.inf])
    max_bounds = np.array([-np.inf, -np.inf, -np.inf])
    
    for file in vtk_files:
        reader = vtk.vtkXMLImageDataReader()
        reader.SetFileName(file)
        reader.Update()
        
        image = reader.GetOutput()
        bounds = image.GetBounds()
        
        min_bounds = np.minimum(min_bounds, bounds[::2])
        max_bounds = np.maximum(max_bounds, bounds[1::2])
    
    return min_bounds, max_bounds

def resample_to_bounds(input_file, output_file, min_bounds, max_bounds):
    """Resample a VTK file to the specified bounding box."""
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(input_file)
    reader.Update()
    
    image = reader.GetOutput()
    resampler = vtk.vtkImageReslice()
    resampler.SetInputData(image)
    
    spacing = image.GetSpacing()
    new_dims = ((max_bounds - min_bounds) / spacing).astype(int) + 1
    
    resampler.SetOutputExtent(0, new_dims[0] - 1, 0, new_dims[1] - 1, 0, new_dims[2] - 1)
    resampler.SetOutputOrigin(min_bounds)
    resampler.SetOutputSpacing(spacing)
    resampler.SetInterpolationModeToNearestNeighbor()
    
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(output_file)
    writer.SetInputConnection(resampler.GetOutputPort())
    writer.Write()

# Directory containing the VTK files
vtk_dir = 'vtk\\31\\'
vtk_files = sorted([os.path.join(vtk_dir, f) for f in os.listdir(vtk_dir) if f.endswith('.vti')])

# Determine the overall bounding box
min_bounds, max_bounds = get_bounding_box(vtk_files)
print(f"Overall bounding box: min_bounds = {min_bounds}, max_bounds = {max_bounds}")

# Resample each file to the bounding box
resampled_dir = 'resampled_vtk\\31\\'
os.makedirs(resampled_dir, exist_ok=True)

for vtk_file in vtk_files:
    output_file = os.path.join(resampled_dir, os.path.basename(vtk_file))
    resample_to_bounds(vtk_file, output_file, min_bounds, max_bounds)

print("Resampling complete. Resampled files saved to", resampled_dir)


Overall bounding box: min_bounds = [0. 0. 0.], max_bounds = [200. 200. 200.]
Resampling complete. Resampled files saved to resampled_vtk\31\
